In [47]:
# Set constants for environment
import os
import sys

PROJECT_NAME = 'lvgl-ui-detector'

IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL_CONTAINER = 'HOMELAB' in os.environ
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/project') # Mount Google Drive
    root_dir = os.path.join("/content/drive/project")
elif IN_LOCAL_CONTAINER:
    root_dir = "/usr/src"
else:
    root_dir = os.path.curdir

In [48]:
if IN_COLAB and 'ultralytics' not in sys.modules:
    %pip install ultralytics
%pip show ultralytics

Name: ultralytics
Version: 8.1.45
Summary: Ultralytics YOLOv8 for SOTA object detection, multi-object tracking, instance segmentation, pose estimation and image classification.
Home-page: 
Author: Glenn Jocher, Ayush Chaurasia, Jing Qiu
Author-email: 
License: AGPL-3.0
Location: /opt/conda/lib/python3.10/site-packages
Editable project location: /usr/src/ultralytics
Requires: matplotlib, opencv-python, pandas, pillow, psutil, py-cpuinfo, pyyaml, requests, scipy, seaborn, thop, torch, torchvision, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Data preparation

In [49]:
# import glob
# if IN_COLAB:
#     repository_dir = os.path.join(root_dir, PROJECT_NAME)
#     dataset_dir = os.path.join(root_dir, 'datasets')
#     dataset_files = glob.glob(os.path.join(dataset_dir, '*.yaml'))
# else:
#     datasets_dir = os.path.join(root_dir, 'tmp')
#     dataset_files = glob.glob(os.path.join(datasets_dir, '*.yaml'))

# print(f"Datasets: {dataset_files}")
from clearml import Dataset
dataset_choice = 'ui_widgets'
target_folder = os.path.join(root_dir, 'dataset')
dataset = Dataset.get(dataset_project='LVGL UI Detector', dataset_name=dataset_choice, auto_create=True, dataset_tags=['lvgl-ui-detector'])
dataset_dir = dataset.get_mutable_local_copy(target_folder, overwrite=True)
dataset_file = os.path.join(dataset_dir, f"{dataset_choice}.yaml")
print(f"Dataset directory: {dataset_dir}")

Dataset directory: /workspace/project/dataset


In [50]:
import yaml

with open(dataset_file, 'r+') as f:
    dataset_content = yaml.safe_load(f)
    dataset_content['path'] = dataset_dir
    print(f"Original dataset:\n{dataset_content}")
    f.seek(0)
    yaml.dump(dataset_content, f)
    f.truncate()
    f.seek(0)
    print(f"Adjusted dataset:\n{f.read()}")
os.listdir(dataset_dir)


Original dataset:
{'names': {0: 'arc', 1: 'bar', 2: 'button', 3: 'buttonmatrix', 4: 'calendar', 5: 'checkbox', 6: 'dropdown', 7: 'label', 8: 'led', 9: 'roller', 10: 'scale', 11: 'slider', 12: 'spinbox', 13: 'switch', 14: 'table', 15: 'textarea'}, 'path': '/workspace/project/dataset', 'test': 'images/test', 'train': 'images/train', 'val': 'images/val'}
Adjusted dataset:
names:
  0: arc
  1: bar
  2: button
  3: buttonmatrix
  4: calendar
  5: checkbox
  6: dropdown
  7: label
  8: led
  9: roller
  10: scale
  11: slider
  12: spinbox
  13: switch
  14: table
  15: textarea
path: /workspace/project/dataset
test: images/test
train: images/train
val: images/val



['labels', 'ui_widgets.yaml', 'images']

# Model training

In [51]:
from clearml import Task, TaskTypes
from ultralytics import YOLO 

# Create a ClearML Task
task = Task.init(
    project_name="LVGL UI Detector",
    task_name=f"train (ui_widgets)",
    task_type=TaskTypes.training,
)

# Load a model
model_variant = "yolov8n"
# Log "model_variant" parameter to task
task.set_parameter("model_variant", model_variant)

# Load the YOLOv8 model
model = YOLO(f'{model_variant}.pt') 

# Put all YOLOv8 arguments in a dictionary and pass it to ClearML
# When the arguments are later changed in UI, they will be overridden here!
args = dict(data=dataset_file, epochs=100, imgsz=480)
task.connect(args)

# Train the model 
# If running remotely, the arguments may be overridden by ClearML if they were changed in the UI
try:
    results = model.train(**args)
except Exception as e:
    raise e
finally:
    task.close()

Jupyter Notebook auto-logging failed, could not access: /workspace/project/lvgl-ui-detector_yolov8-jvsc-85350928-bb0c-430e-b34b-b9ddbfd9b21fad5ae19c-9704-4cb4-b817-3a8206d876d9.ipynb


ClearML Task: overwriting (reusing) task id=fadfb35fb6b04861b3c01263770b45fa
ClearML results page: https://app.clear.ml/projects/5606a5f963624e2991df71add8ec09ce/experiments/fadfb35fb6b04861b3c01263770b45fa/output/log
Ultralytics YOLOv8.1.45 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/workspace/project/dataset/ui_widgets.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sou

train: Scanning /workspace/project/dataset/labels/train... 534 images, 0 backgrounds, 229 corrupt: 100%|██████████| 534/534 [00:00<00:00, 632.05it/s]

train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_0.jpg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 16. Possible class labels are 0-15
train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_1.jpg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 16. Possible class labels are 0-15
train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_10.jpg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 16. Possible class labels are 0-15
train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_13.jpg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 16. Possible class labels are 0-15
train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_14.jpg: ignoring corrupt image/label: Label class 17 exceeds dataset class count 16. Possible class labels are 0-15
train: WARNING ⚠️ /workspace/project/dataset/images/train/roller_15.jpg: ignoring corrupt image/l

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


100 epochs completed in 0.159 hours.
Optimizer stripped from /usr/src/ultralytics/runs/detect/train13/weights/last.pt, 6.2MB
Optimizer stripped from /usr/src/ultralytics/runs/detect/train13/weights/best.pt, 6.2MB

Validating /usr/src/ultralytics/runs/detect/train13/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.14 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 2060 with Max-Q Design, 6144MiB)
Model summary (fused): 168 layers, 3008768 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.74it/s]

: 

: 

: 

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

results = model.train(data=f"/workspace/project/tmp/random_widgets.yaml", epochs=100, imgsz=480, device=[0])

In [ ]:
# Perform validation using the validate function
model = YOLO('/usr/src/ultralytics/runs/detect/train7/weights/best.pt')
results = model.val()

In [ ]:
results.box.map

In [ ]:
results.box.map50

In [ ]:
results.box.map75

In [ ]:
results.box.maps

In [ ]:
from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
benchmark(model='/usr/src/ultralytics/runs/detect/train7/weights/best.pt', data='/workspace/project/tmp/custom/custom.yaml', imgsz=416, half=False, device=[0])